Source of data if you messed up the data: [Kaggle](https://www.kaggle.com/datasets/hereisburak/pins-face-recognition?datasetId=543939&sortBy=voteCount)

In [20]:
import os
import re
import shutil
import random

In [23]:
# Function to create new folder not the same as the original
# Original folder has something like pin_{name_of_celebrities}, we need this function to make it readable
# Source of data is above if you messed up!

origin_path = "../data/imgs_face_original"
to_path = "../data/imgs_face_clean_original"

def clean_dir(original_dir, clean_dir):
    """_summary_

    Args:
        original_dir (_type_): Original Dir with name pin_
        clean_dir (_type_): Dir that you want to put all image in

    Example:
        origin_path = "../data/imgs_face_original"
        to_path = "../data/imgs_face_clean_original"
        clean_dir(original_dir=origin_path, clean_dir=to_path)
    """
    if  os.path.exists(clean_dir):
        shutil.rmtree(clean_dir)
    else: os.makedirs(clean_dir)

    total_folders = len(os.listdir(original_dir))
    copies_folders = 0

    for foldername in os.listdir(original_dir):
        if os.path.isdir(os.path.join(original_dir, foldername)):
            # copy the folder to `clean_dir`
            shutil.copytree(os.path.join(original_dir, foldername), os.path.join(clean_dir, foldername))
            new_foldername = re.sub(r"pins_","", foldername)
            new_foldername = new_foldername.replace(" ", "_")
            # rename the folder with the new foldername
            os.rename(os.path.join(clean_dir, foldername), os.path.join(clean_dir, new_foldername))
            copies_folders += 1
            print(f"Progress: {copies_folders}/{total_folders}", end="\r")

clean_dir(original_dir=origin_path, clean_dir=to_path)


In [38]:
import random

origin = "../data/imgs_face_clean/"
model = "../data/for_model"

def dir_prep_train_test(origin_dir:str, for_model_dir:str, train_size:int=0.8):
    """To split the images of people in seperate directory to train, test, and valid

    Args:
        origin_dir (_type_): Directory with image that already have a photo of several people but doesn't split
        for_model_dir (_type_): Directory that you want to put the data to train, test, val

    Example:
        dir_prep_train_test(origin_dir="../data/imgs_face_clean/", for_model_dir="../data/for_model")
    """


    train_dir = for_model_dir + "/train"
    test_dir = for_model_dir + "/test"
    val_dir = for_model_dir + "/val"


    dirs = [train_dir, test_dir, val_dir]
    for dir in dirs:
        if not os.path.exists(dir):
            os.makedirs(dir)
    
    total_folders = len(os.listdir(origin_dir))
    processed_folders = 0

    for subdir in os.listdir(origin_dir):
        if os.path.isdir(os.path.join(origin_dir, subdir)):
            # Get a list of image
            images = [f for f in os.listdir(os.path.join(origin_dir, subdir)) if f.endswith(".jpg")]
            random.shuffle(images)

            # Split the images into train and test
            test_size = (1-train_size)/2
            val_size = (1-train_size)/2
            if train_size+test_size+val_size != 1:
                print("Train ratio not compatible")
            num_train, num_test, num_val = int(len(images)*train_size), int(len(images)*test_size), int(len(images)*val_size)
            train_images = images[0:num_train]
            test_images = images[num_train:num_train+num_test]
            valid_images = images[num_train+num_test:]

            # Create the subfolders in train, test, and valid dirs
            os.makedirs(os.path.join(train_dir, subdir), exist_ok=True)
            os.makedirs(os.path.join(test_dir, subdir), exist_ok=True)
            os.makedirs(os.path.join(val_dir, subdir), exist_ok=True)

            # Move the images to the subdirs
            for image in train_images:
                shutil.copy2(os.path.join(origin_dir, subdir, image), os.path.join(train_dir, subdir, image))
            for image in test_images:
                shutil.copy2(os.path.join(origin_dir, subdir, image), os.path.join(test_dir, subdir, image))
            for image in valid_images:
                shutil.copy2(os.path.join(origin_dir, subdir, image), os.path.join(val_dir, subdir, image))

            processed_folders += 1
            print(f"Progress: {processed_folders}/{total_folders}", end="\r")

dir_prep_train_test(origin_dir=origin, for_model_dir=model)